<div style="font-size:2em;text-align:left">Videx: Counting and categorizing vehicles in surveillance</div>

* [문제 소개](#1)
* [데이터 처리 과정](#2)
* [다뤄본 모델 소개 및 성능 향상의 과정](#3)
    * [성능에 영향을 미친 요인 분석/설명](#4)   

<a id="1"></a>
### 📍 문제 소개

제공된 교통량정보제공시스템의 CCTV 데이터셋을 사용해 **12개 차종별로 교통량을 정확하게 측정하는 모델**을 개발

**데이터 설명**

- **출처**: 한국건설기술연구원
- **수집 방식**: 교통량정보제공시스템(TMS)의 CCTV 영상에서 추출된 이미지
- **데이터 기간**: 2019~2020년 '일반국도 수시차종분류조사' 동영상 자료
- **데이터 특징**:
  - 총 5,585장의 이미지 데이터
  - 개인정보 보호를 위해 사람의 얼굴과 차량의 번호판은 모자이크 처리되어 있음
  - 이미지 내 차량 객체 인식을 위한 2D 객체 인식(Object Detection) 기술 개발에 활용 가능

**데이터 분류**

- **훈련 데이터 (Train Data)**: 4,462장
- **테스트 데이터 (Test Data)**: 1,123장
- 데이터는 8:2의 비율로 훈련 데이터와 테스트 데이터로 나누어 제공됨

**대회 목표**

제공된 이미지를 사용하여 차량을 12개 차종으로 분류

**평가 기준**

모델이 12개 차종을 얼마나 정확하게 분류하고 교통량을 측정하는지에 따라 평가, FScoreMacro 사용

**데이터셋 클래스**

![데이터셋 클라스](data/data_class.png)


<a id="2"></a>
### 📍 데이터 처리 과정

#데이터처리 과정

**데이터 파악**
- 데이터 셋은 이미지 데이터 셋이며, 세부 정보가 포함된 세부 xml 파일이 포함된 5,585개의 이미지 데이터 셋으로 구성

    - <이미지 데이터>    
        - train - the training set (4,462 train data set (image/xml files))
        - test - the test set (1,123 test data set (image))
        - Class - 12
    
    - <어노테이션 데이터> - 이미지 데이터에 대한 레이블
        - < filename > 파일명 (고유 ID)
        - < source (database, annotation, image) >, < size (width, height, depth) >, < segmented >
        - < name > - 차량의 종류 (01~12) ※차량의 대수만큼 존재
        - < bndbox(xmin, ymin, xmax, ymax) >

1. 데이터 분리

    ``` kf = KFold(n_splits=5, shuffle=True, random_state=42) ```
    - Cross Validation(교차 검증) 
        - 부족한 데이터 개수
            - 훈련 데이터 셋을 검증 데이터 셋으로 나누기엔 부족
        
        - 훈련 및 평가 성능 
            - 모든 데이터 셋을 평가에 활용 가능 -> 과적합(Overfitting) 방지 / 데이터 편중 방지
            - 모든 데이터 셋을 훈련에 활용 가능 -> 정확도 향상 / 데이터 부족에서 오는 과소적합(Underfitting) 방지

        - 일반화
            - 모든 데이터가 최소 한번씩 훈련이 되어 모델이 더 잘 일반화 됨
            ```
            folds/
                fold_0/
                    images/
                    labels/
                    data.yaml
                fold_1/
                fold_2/
                fold_3/
            ```

2. 하이퍼파라미터 조정   
    ``` '--hyp /home/work/Yongbin/Videx/code/yolo5/data/hyps/hyp.scratch-low.yaml' ```
    
    - 사전 구성된 하이퍼파라미터 사용 및 일부 조정(낮은 수준의 데이터 증강 설정.)

    - 파인튜닝 < 사전 학습된 모델을 새로운 데이터셋에 맞춰 재학습하여 성능을 최적화>  
        - 학습 파라미터(weight와 bias) 불러오기  
            ```yolov5(s,m,l,x).pt   ----- '--weights', 'yolov5l.pt'```
        - lr-> 기존 0.1 -> 수정 0.001
        - weight_decay: 0.0005  (가중치가 너무 커지는 것을 방지하여 과적합을 줄이는 역할)
    - 데이터 증강 <데이터의 다양성을 인위적으로 증가시켜 모델의 일반화 성능을 향상>
        ```
        hsv_h: 0.015 - 이미지의 HSV-Hue 채널을 조정 (색조)
        hsv_s: 0.7 - 이미지의 HSV-Saturation 채널을 조정 (채도)
        hsv_v: 0.4 - 이미지의 HSV-Value 채널을 조정 (명도)
        translate: 0.1 
        scale: 0.5 
        ....
        ```


In [1]:
import subprocess

def train_fold(fold):
    print(f"Training fold {fold}")
    process = subprocess.Popen([
        'python', 'yolov5/train.py',
        '--img', '1024',
        '--batch', '32',
        '--epochs', '300',
        '--data', f'/home/work/Yongbin/Videx/data/folds/fold_{fold}/data.yaml',
        '--cfg', 'yolov5/models/yolov5l.yaml',  
        '--weights', 'yolov5l.pt',              
        '--name', f'yolov5_vehicle_detection_fold_{fold}',
        '--hyp /home/work/Yongbin/Videx/code/yolo5/data/hyps/hyp.scratch-low.yaml'
        '--device', '0'  
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    

<a id="3"></a>
### 📍 다뤄본 모델 소개 및 성능 향상의 과정

**1. CNN**

- 초반 모델을 설계할 때 Convolutional Neural Network(CNN)을 사용하여 12개의 차종 학습 시도
- 모델 학습 시 CNN을 사용, 테스트 시 Yolov5를 사용하여 차량 detect 후 예측 실행
- CNN 모델로는 EfficientNet 사용
- 학습 데이터셋은 각각 차량을 Crop 후 (224,224)로 resize
![학습 데이터 전처리](data/train_crop_resize_224.png)

- 테스트 데이터셋은 각각 차량을 YOLO를 사용하여 detect
![테스트 데이터셋 검출](data/yolo_test.jpeg)


**2. YOLO**

- 이전방식으로는 모델 test 과정이 두 단계로 나뉨
    - YOLO를 사용해 test 데이터에서 차를 detect
    - 검출된 object를 crop후 train된 모델에 test
- 하지만 위와같은 방식으로는 모델 성능이 높지 않았음
    - Train과 Test과정 모두 개선이 필요하다 느낌
- Train을 처음부터 YOLO 모델을 사용해 학습시키면 12개의 클래스로 구분 가능
- 그러면 test도 crop 및 별도의 과정을 거치지 않아도 곧바로 결과를 얻을 수 있음
- YOLOv5s 모델에 epoch 20으로 train과 validation 데이터를 9:1로 나누고 학습 진행
    - 전에 비해 점수 0.863으로 성능 대폭 향상
    - Epoch 수 늘리기 + 데이터 전처리를 통해 추가적인 모델 개선
- 이후에 Cross Validation 및 Hyperparameter 튜닝으로 모델 개선



**성능 비교**

**CNN or YOLO**  

![CNN/YOLO 성능 비교](data/Train_CNN_Test_Yolo.png)

<a id="4"></a>
#### 📍 성능에 영향을 미친 요인 분석/설명

**YOLO Fine-tuning**  

![YOLO 성능 비교](data/Yolo_only.png)

---

**YOLO Fine-tuning & Hyperparameter tuning**  

![YOLO 성능 비교](data/yolo_hyper.png)

---